In [1]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf
import scipy.stats as scs
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
df = pd.read_csv("GlobalLandTemperaturesByCountry.csv")

In [3]:
df.isnull().sum()

dt                                   0
AverageTemperature               32651
AverageTemperatureUncertainty    31912
Country                              0
dtype: int64

In [4]:
df = df.dropna()

In [5]:
df.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
Country                           object
dtype: object

In [6]:
df.dt = pd.to_datetime(df.dt)

In [7]:
df

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
5,1744-04-01,1.530,4.680,Åland
6,1744-05-01,6.702,1.789,Åland
7,1744-06-01,11.609,1.577,Åland
8,1744-07-01,15.342,1.410,Åland
...,...,...,...,...
577456,2013-04-01,21.142,0.495,Zimbabwe
577457,2013-05-01,19.059,1.022,Zimbabwe
577458,2013-06-01,17.613,0.473,Zimbabwe
577459,2013-07-01,17.000,0.453,Zimbabwe


In [8]:
ts = pd.read_csv("ppm_ts.csv")
ts = ts.rename({'Unnamed: 0': 'dt'}, axis=1)
ts.dt = pd.to_datetime(ts.dt)


In [9]:
df2 = df.merge(ts, on='dt', how='left')
df2 = df2.dropna()
df2

,dt,AverageTemperature,AverageTemperatureUncertainty,Country,co2
2500,1958-03-01,-5.797,0.351,Åland,316.100000
2501,1958-04-01,0.361,0.383,Åland,317.200000
2502,1958-05-01,6.064,0.590,Åland,317.433333
2503,1958-06-01,11.018,0.595,Åland,315.625000
2504,1958-07-01,14.556,0.579,Åland,315.625000
...,...,...,...,...,...
544806,2013-04-01,21.142,0.495,Zimbabwe,396.740000
544807,2013-05-01,19.059,1.022,Zimbabwe,396.750000
544808,2013-06-01,17.613,0.473,Zimbabwe,395.960000
544809,2013-07-01,17.000,0.453,Zimbabwe,394.470000


In [10]:
grouped = df2.groupby(df2.Country)

In [11]:
countries = df2.Country.unique()

In [12]:
grouped.get_group("United States")

,dt,AverageTemperature,AverageTemperatureUncertainty,Country,co2
525986,1958-03-01,1.449,0.160,United States,316.100000
525987,1958-04-01,8.144,0.236,United States,317.200000
525988,1958-05-01,14.859,0.301,United States,317.433333
525989,1958-06-01,18.729,0.335,United States,315.625000
525990,1958-07-01,21.098,0.348,United States,315.625000
...,...,...,...,...,...
526648,2013-05-01,14.073,0.178,United States,396.750000
526649,2013-06-01,20.198,0.236,United States,395.960000
526650,2013-07-01,22.074,0.152,United States,394.470000
526651,2013-08-01,21.168,0.249,United States,393.240000


In [13]:
f_ts = pd.read_csv("fc_all.csv")
f_ts = f_ts.rename({'Unnamed: 0': 'dt'}, axis=1)
f_ts.dt = pd.to_datetime(f_ts.dt)

In [14]:
f_ts

,dt,forecast,lower_ci_95,lower_ci_99,upper_ci_95,upper_ci_99
0,2021-09-01,413.937732,413.403799,413.403799,414.471666,414.471666
1,2021-10-01,415.439453,414.647809,414.647809,416.231097,416.231097
2,2021-11-01,416.393342,415.409311,415.409311,417.377372,417.377372
3,2021-12-01,417.084395,415.939873,415.939873,418.228918,418.228918
4,2022-01-01,417.612747,416.327622,416.327622,418.897873,418.897873
...,...,...,...,...,...,...
495,2062-12-01,514.581050,401.977827,401.977827,627.184273,627.184273
496,2063-01-01,515.109385,402.183005,402.183005,628.035766,628.035766
497,2063-02-01,515.553522,402.304906,402.304906,628.802139,628.802139
498,2063-03-01,515.919420,402.349482,402.349482,629.489357,629.489357


In [15]:
f_ts2 = pd.read_csv("fc_all2.csv")
f_ts2 = f_ts2.rename({'Unnamed: 0': 'dt'}, axis=1)
f_ts2.dt = pd.to_datetime(f_ts2.dt)
f_ts2.head(5)

,dt,forecast,lower_ci_95,lower_ci_99,upper_ci_95,upper_ci_99
0,2021-01-01,3.863577e+10,3.696842e+10,3.696842e+10,4.030311e+10,4.030311e+10
1,2022-01-01,3.778710e+10,3.513623e+10,3.513623e+10,4.043798e+10,4.043798e+10
2,2023-01-01,3.827615e+10,3.491843e+10,3.491843e+10,4.163387e+10,4.163387e+10
3,2024-01-01,3.857805e+10,3.463833e+10,3.463833e+10,4.251777e+10,4.251777e+10
4,2025-01-01,3.851044e+10,3.406426e+10,3.406426e+10,4.295662e+10,4.295662e+10


In [16]:
f_ts2["cumulative"] = 0
sum = 0
for i in range(len(f_ts2)):
    sum += f_ts2.iloc[i,1]
    f_ts2.iloc[i,6] = sum
f_ts2.head(5)

,dt,forecast,lower_ci_95,lower_ci_99,upper_ci_95,upper_ci_99,cumulative
0,2021-01-01,3.863577e+10,3.696842e+10,3.696842e+10,4.030311e+10,4.030311e+10,3.863577e+10
1,2022-01-01,3.778710e+10,3.513623e+10,3.513623e+10,4.043798e+10,4.043798e+10,7.642287e+10
2,2023-01-01,3.827615e+10,3.491843e+10,3.491843e+10,4.163387e+10,4.163387e+10,1.146990e+11
3,2024-01-01,3.857805e+10,3.463833e+10,3.463833e+10,4.251777e+10,4.251777e+10,1.532771e+11
4,2025-01-01,3.851044e+10,3.406426e+10,3.406426e+10,4.295662e+10,4.295662e+10,1.917875e+11


In [21]:
import math
def calc_tempature(country, year, month, option1 = 0,option2 = 0,option3 = 0,option4 = 0,option5 = 0,option6 = 0):
    df = grouped.get_group(country)
    date = str(year) + "-" + str(month)
    ref_year = "2010"
    print(f"Predicted Country             : {country}")
    print(f"Predicted Date                : {date}")
    expected_ppm = float(f_ts[f_ts["dt"] == date].forecast)
    ref_temp = float(df[df["dt"] == ref_year +"-"+ month].AverageTemperature)
    ref_ppm  = float(df[df["dt"] == ref_year +"-"+ month].co2)
    
    #FORMULA
    expected_temp = ref_temp + 3*math.log2(expected_ppm/ref_ppm)
    #CHANGES
    expected_emission = float(f_ts2[f_ts2.dt == year].cumulative)
    sum = 0
    sum += (int(option1)/10)*(25/100)*expected_emission
    sum += (int(option2)/10)*(24/100)*expected_emission
    sum += (int(option3)/10)*(6.4/100)*expected_emission
    sum += (int(option4)/10)*(14/100)*expected_emission
    sum += (int(option5)/10)*(21/100)*expected_emission
    sum += (int(option6)/10)*(9.6/100)*expected_emission

    ppm_change = sum/2300000000
    expected_ppm_ac = expected_ppm - ppm_change

    #FORMULA
    expected_temp_ac = ref_temp + 3*math.log2(expected_ppm_ac/ref_ppm)


    print(f"Reference tempature in {country}    : {round(ref_temp,2)}")
    print(f"Reference CO2 PPM in Worldwide      : {ref_ppm}")
    print(f"Expected CO2 PPM in Worldwide       : {round(expected_ppm, 2)}")
    print(f"Expected tempature in {country}     : {round(expected_temp, 2)}")
    print(f"Expected cumulative CO2 emission in Worldwide from 2021 to {year}  : {expected_emission:,}")
    print(f"Expected cumulative CO2 emission in Worldwide from 2021 to {year} after changes :{expected_emission-sum:,}")
    print(f"Expected tempature in {country} after changes : {round(expected_temp_ac, 2)}")
    print(f"Expected CO2 PPM in Worldwide after changes      : {round(expected_ppm_ac, 2)}")
    print("\nSummary of the impact of the changes to be made (Expected Value - Expected Value After Changes)\n")
    print(f"PPM Change in {int(year)-2021} year : {round(expected_ppm-expected_ppm_ac,2)}")
    print(f"CO2 Emission Change in {int(year)-2021} year : {round(sum,2):,}")
    print(f"Tempature Change in {int(year)-2021} year : {round(expected_temp-expected_temp_ac,2)}")


In [22]:
def app():
    print("Welcome to the Climate Change Predictor App\n")
    print("""In order to be able to predict future temperature changes,\nwe will foresee how it will change in which sectors that affect CO2 emission on a global scale,\nif we switch to clean technologies in them.
""")
    print("""*In this study, historical CO2 concentration and CO2 emission data (1900-2020) were analyzed by ARIMA and forecasted until 2063.""")
    print("*2010 global CO2 emission data were taken as reference to model the effects of sectors on cumulative CO2 emission values.")
    print("""*Based on estimates of the total quantity of CO2 in the atmosphere (in gigatons, abbreviated GtC) and of the CO2 concentration, every 2.3 GtC of emissions would be expected to raise atmospheric CO2 concentration by 1 ppm.""")
    print("\nModel : T = T0 + S*log2 (C / C0)\n")
    print("""\tT is the new/current temperature\n\tT0 is the know temperature at some reference time\n\tS is the "climate sensitivity" factor; we've been using 3° C According to the Fourth Assessment Report by the IPCC (Intergovernmental Panel on Climate Change)
\tC is the new/current atmospheric CO2 concentration\n\tC0 is the known atmospheric CO2 concentration at some reference time (2010)\n""")
    print("0-5 Scale\n0 : No Change in Technology\n5 : All Green Revolution\n")
    print("Electricity and Heat Production")
    option1 = input()
    print(option1)
    print("Agriculture-Forestry-Other Land Use")
    option2 = input()
    print(option2)
    print("Buildings")
    option3 = input()
    print(option3)
    print("Transport")
    option4 = input()
    print(option4)
    print("Industry")
    option5 = input()
    print(option5)
    print("Other Energy")
    option6 = input()
    print(option6)
    print("Select Country:")
    country = input()
    print(country)
    print("Select Year:")
    year = input()
    print(year)
    print("Select Month:")
    month = input()
    print(month)
    calc_tempature(country,year, month, option1,option2,option3,option4,option5,option6)

In [24]:
app()

Welcome to the Climate Change Predictor App

In order to be able to predict future temperature changes,
we will foresee how it will change in which sectors that affect CO2 emission on a global scale,
if we switch to clean technologies in them.

*In this study, historical CO2 concentration and CO2 emission data (1900-2020) were analyzed by ARIMA and forecasted until 2063.
*2010 global CO2 emission data were taken as reference to model the effects of sectors on cumulative CO2 emission values.
*Based on estimates of the total quantity of CO2 in the atmosphere (in gigatons, abbreviated GtC) and of the CO2 concentration, every 2.3 GtC of emissions would be expected to raise atmospheric CO2 concentration by 1 ppm.

Model : T = T0 + S*log2 (C / C0)

	T is the new/current temperature
	T0 is the know temperature at some reference time
	S is the "climate sensitivity" factor; we've been using 3° C According to the Fourth Assessment Report by the IPCC (Intergovernmental Panel on Climate Change)
	C